# <center>Genre Classification Using Extracted Audio Features</center>

<center> Levi Davis <br> ljd3frf@virginia.edu <br> CS 6501: Digital Signal Processing, Spring 2023 </center>

In [1]:
%matplotlib inline

import os
import shutil
import time

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
from PIL import Image
import scipy as stats 
import warnings
from pretty_html_table import build_table
import random

from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

plt.rcParams['figure.figsize'] = (17, 5)

# Preprocessing

In [ ]:
# Reformat data folder
# Puts all songs into one folder

# Define the source directory where the subfolders are located
source_dir = os.getcwd() + '/data/fma_small/fma_small'

# Define the destination directory where you want the files to be moved
destination_dir = os.getcwd() +'/data/data_small'

# Loop through all the subdirectories in the source directory
for subdir in os.listdir(source_dir):
    # Define the full path to the subdirectory
    subdir_path = os.path.join(source_dir, subdir)
    # Check if the item in the directory is a directory
    if os.path.isdir(subdir_path):
        # Loop through all the files in the subdirectory
        for file in os.listdir(subdir_path):
            # Define the full path to the file
            file_path = os.path.join(subdir_path, file)
            
            # Move the file to the destination directory
            shutil.move(file_path, destination_dir)
        
        # Remove the empty subdirectory
        os.rmdir(subdir_path)

# Feature Extraction

In [4]:
# modeled from https://github.com/mdeff/fma/blob/master/features.py

def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rms=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1, tempogram_ratio=13)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def compute_features(tid, filepath):

    features = pd.Series(index=columns(), dtype=np.float32, name=tid)


    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        x, sr = librosa.load(filepath, sr=None, mono=True)  # kaiser_fast

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)
        
        f = librosa.feature.tempogram_ratio(y=x)
        feature_stats('tempogram_ratio', f)
        
        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rms(S=stft)
        feature_stats('rms', f)
        

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)
        
        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)

    except Exception as e:
        print('{}: {}'.format(tid, repr(e)))

    return features


In [ ]:
# Directory where mp3 are stored.
DATA_DIR = '\\data\\data_small\\'
DATA_PATH = os.getcwd() + DATA_DIR
song_files = os.listdir(DATA_PATH)

# Loop through all song files, compute all feature info, and store df as csv
series_list = []
for file in song_files:
    track_id = file[-10:-4]
    file_path = os.path.join(DATA_PATH, file)
    feature_series = compute_features(track_id, file_path)
    series_list.append(feature_series)
    
DF = pd.DataFrame(series_list)

# Save to csv
DF.to_csv('computed_features.csv')

# Modeling

## Prep Data

In [7]:
# Load data from csv
SONG_DATA = pd.read_csv('computed_features.csv',header=[0,1,2], index_col= 0)
SONG_DATA.index.name = 'track_id'
# Remove songs with NAN values
SONG_DATA.dropna(inplace=True)
print(SONG_DATA.shape)

(7951, 609)


In [8]:
track_info_dir = os.getcwd() + '/data/fma_metadata/tracks.csv'
tracks = pd.read_csv(track_info_dir, index_col=0, header=[0,1])

# using small dataset of 8,000 songs
track_small = tracks[tracks['set', 'subset'] == 'small']
genres = track_small['track','genre_top']
genres.name = 'genre'

In [10]:
## Make sure features and labels obs are equal
# get the indices that are not present in both the dataframe and the series
idx_to_drop = SONG_DATA.index.symmetric_difference(genres.index)
# drop the rows from the dataframe
try:
    SONG_DATA.drop(idx_to_drop, inplace=True)
except:
    pass
# drop the rows from the labels
try:
    genres.drop(idx_to_drop, inplace=True)
except:
    pass

assert genres.shape[0] == SONG_DATA.shape[0]

### Models with one feature

In [33]:
RESULTS = pd.DataFrame(columns= ['classifier', 'feature', 'test_accuracy', 'duration'])

classifiers = [SVC(), RandomForestClassifier(), KNeighborsClassifier(), 
               DecisionTreeClassifier(), GradientBoostingClassifier()]
classifier_names = ['SVC', 'RandomForest', 'KNN', 'DecisionTree', 'GradientBoosting']

features = SONG_DATA.columns.levels[0].values
for feature in features:
    # Separate the data into features and labels
    X = SONG_DATA[feature]
    y = genres
    # Split the data into train, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, 
                                                        random_state=51, stratify=y)
    
    # Be sure training samples are shuffled.
    X_train, y_train = skl.utils.shuffle(X_train, y_train, random_state=0)

    # Standardize features by removing the mean and scaling to unit variance.
    scaler = skl.preprocessing.StandardScaler()
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    
    for i,cl in enumerate(classifiers):
        # Support vector classification.
        clf = cl
        start = time.time() # start timer
        clf.fit(X_train, y_train)
        stop = time.time() # stop time
        duration = stop-start # compute time to run algo
        score = clf.score(X_test, y_test)

        # Append the results to the DataFrame
        RESULTS.loc[len(RESULTS)] = [classifier_names[i], feature, score, duration]


RESULTS.to_csv('ML_single_feature_classification_results.csv'. index=False)

### Models with two features

In [39]:
RESULTS_PAIRS = pd.DataFrame(columns= ['classifier', 'feature_pair', 'test_accuracy','duration'])

classifiers = [SVC(), RandomForestClassifier(), KNeighborsClassifier(), 
               DecisionTreeClassifier(), GradientBoostingClassifier()]
classifier_names = ['SVC', 'RandomForest', 'KNN', 'DecisionTree', 'GradientBoosting']

features = SONG_DATA.columns.levels[0].values
pairs = [[a, b] for idx, a in enumerate(features) for b in features[idx + 1:]]

for feature_pair in pairs:
    # Separate the data into features and labels
    X = SONG_DATA[feature_pair]
    y = genres
    # Split the data into train, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, 
                                                        random_state=51, stratify=y)
    
    # Be sure training samples are shuffled.
    X_train, y_train = skl.utils.shuffle(X_train, y_train, random_state=0)

    # Standardize features by removing the mean and scaling to unit variance.
    scaler = skl.preprocessing.StandardScaler()
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    
    for i,cl in enumerate(classifiers):
        # Support vector classification.
        clf = cl
        start = time.time() # start timer
        clf.fit(X_train, y_train)
        stop = time.time() # stop time
        duration = stop-start # compute time to run algo
        score = clf.score(X_test, y_test)

        # Append the results to the DataFrame
        RESULTS_PAIRS.loc[len(RESULTS_PAIRS)] = [classifier_names[i], feature_pair, score, duration]


RESULTS_PAIRS.to_csv('ML_two_feature_classification_results.csv', index =False)

### Models with all features

In [36]:
RESULTS_ALL = pd.DataFrame(columns= ['classifier', 'test_accuracy', 'duration'])

classifiers = [SVC(), RandomForestClassifier(), KNeighborsClassifier(), 
               DecisionTreeClassifier(), GradientBoostingClassifier()]
classifier_names = ['SVC', 'RandomForest', 'KNN', 'DecisionTree', 'GradientBoosting']


# Separate the data into features and labels
X = SONG_DATA
y = genres
# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51, stratify=y)

# Be sure training samples are shuffled.
X_train, y_train = skl.utils.shuffle(X_train, y_train, random_state=0)

# Standardize features by removing the mean and scaling to unit variance.
scaler = skl.preprocessing.StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

for i,cl in enumerate(classifiers):
    # Support vector classification.
    clf = cl
    start = time.time() # start timer
    clf.fit(X_train, y_train)
    stop = time.time() # stop time
    duration = stop-start # compute time to run algo
    score = clf.score(X_test, y_test)

    # Append the results to the DataFrame
    RESULTS_ALL.loc[len(RESULTS_ALL)] = [classifier_names[i], score, duration]
    
    
RESULTS_ALL.to_csv('ML_all_features_classification_results.csv',index=False)

In [ ]:
RESULTS_ALL

# Second round: model hyperparamter tuning

In [19]:
# top two classifiers
classifiers = [GradientBoostingClassifier(), RandomForestClassifier()]
classifier_names = ['Gradient Boosting', 'Random Forest']
# top 2 single features and top 4 feature pairs
features = ['mfcc', 'spectral_contrast', ['mfcc', 'spectral_contrast'], ['mfcc', 'chroma_cqt'],
           ['mfcc', 'zcr'], ['mfcc', 'tempogram_ratio']]

RESULTS = pd.DataFrame(columns= ['classifier', 'feature_pair', 'test_accuracy','duration'])

for feature_pair in features:
    # Separate the data into features and labels
    X = SONG_DATA[feature_pair]
    y = genres
    # Split the data into train, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, 
                                                        random_state=51, stratify=y)
    
    # Be sure training samples are shuffled.
    X_train, y_train = skl.utils.shuffle(X_train, y_train, random_state=0)

    # Standardize features by removing the mean and scaling to unit variance.
    scaler = skl.preprocessing.StandardScaler()
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    
    for i,cl in enumerate(classifiers):
        clf = cl
        # start timer
        start = time.time() # start timer
        # Perform 5-fold cross-validation on the training set
        cv_scores = cross_val_score(gbc, X_train, y_train, cv=5)

        clf.fit(X_train, y_train)
        stop = time.time() # stop time
        duration = stop-start # compute time to run algo
        score = clf.score(X_test, y_test)
        # Append the results to the DataFrame
        RESULTS.loc[len(RESULTS)] = [classifier_names[i], feature_pair, score, duration]


RESULTS.to_csv('HPCV_classification_results.csv', index =False)

# Make tables

### Single features

In [107]:
sf = pd.read_csv('ML_single_feature_classification_results.csv',index_col=0)\
            .sort_values('test_accuracy', ascending = False)\
            .reset_index().drop(columns='index')

In [108]:
single_feature_table = build_table(sf, 'blue_light', font_size='18px', index=True)

In [109]:
# save table
with open('Images/single_feature_table.html', 'w') as f:
    f.write(single_feature_table)

In [110]:
single_feature_table_head = build_table(sf.head(10), 'blue_light', font_size='18px', index=True)

In [111]:
# save table
with open('Images/single_feature_table_head.html', 'w') as f:
    f.write(single_feature_table_head)

In [112]:
sf_features = sf.groupby('feature').mean().sort_values('test_accuracy',ascending=False).reset_index()

C:\Users\ljd3frf\AppData\Local\Temp\ipykernel_8272\409683886.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sf_features = sf.groupby('classifier').mean().sort_values('test_accuracy',ascending=False).reset_index()


In [114]:
single_feature_table_features = build_table(sf_features, 'blue_light', font_size='18px', index=True)
# save table
with open('Images/single_feature_table_features.html', 'w') as f:
    f.write(single_feature_table_features)

In [ ]:
sf_algos = sf.groupby('classifier').mean().sort_values('test_accuracy',ascending=False).reset_index()

In [129]:
single_feature_table_algos = build_table(sf_algos, 'blue_light', font_size='18px', index=True)
# save table
with open('Images/single_feature_table_algos.html', 'w') as f:
    f.write(single_feature_table_algos)

### Two features

In [115]:
df = pd.read_csv('ML_two_feature_classification_results.csv')\
            .sort_values('test_accuracy', ascending = False)\
            .reset_index().drop(columns='index')

In [117]:
two_feature_table = build_table(df, 'blue_light', font_size='18px', index=True)

In [118]:
# save table
with open('Images/two_feature_table.html', 'w') as f:
    f.write(two_feature_table)

In [ ]:
two_feature_table_head = build_table(df.head(10), 'blue_light', font_size='18px', index=True)

In [120]:
# save table
with open('Images/two_feature_table_head.html', 'w') as f:
    f.write(two_feature_table_head)

In [139]:
df_features = df.groupby('feature_pair').mean().sort_values('test_accuracy',ascending=False).reset_index()

C:\Users\ljd3frf\AppData\Local\Temp\ipykernel_8272\108149541.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_features = df.groupby('feature_pair').mean().sort_values('test_accuracy',ascending=False).reset_index()


In [140]:
two_feature_table_features = build_table(df_features.head(10), 'blue_light', font_size='18px', index=True)

In [149]:
# save table
with open('Images/two_feature_table_features.html', 'w') as f:
    f.write(two_feature_table_features)

In [155]:
df_algos = df.groupby('classifier').mean().sort_values('test_accuracy',ascending=False).reset_index()

C:\Users\ljd3frf\AppData\Local\Temp\ipykernel_8272\2726571204.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_algos = df.groupby('classifier').mean().sort_values('test_accuracy',ascending=False).reset_index()


In [156]:
two_feature_table_algos = build_table(two_feature_table_algos.head(10), 'blue_light', font_size='18px', index=True)

In [157]:
# save table
with open('Images/two_feature_table_algos.html', 'w') as f:
    f.write(two_feature_table_algos)

### All features

In [121]:
af = pd.read_csv('ML_all_features_classification_results.csv')\
            .sort_values('test_accuracy', ascending = False)\
            .reset_index().drop(columns='index')

In [122]:
all_feature_table = build_table(af, 'blue_light', font_size='18px', index=True)

In [123]:
# save table
with open('Images/all_feature_table.html', 'w') as f:
    f.write(all_feature_table)

### Tuned models

In [29]:
tm = pd.read_csv('HPCV_classification_results.csv',index_col=0)\
            .sort_values('test_accuracy', ascending = False)\
            .reset_index()

In [30]:
tm__table = build_table(tm, 'blue_light', font_size='18px', index=True)

In [31]:
# save table
with open('Images/tm__table.html', 'w') as f:
    f.write(tm__table)

# CNN

In [10]:
# TensorFlow
import tensorflow as tf
from tensorflow.keras import utils, initializers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, save_model, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.layers import AveragePooling2D


## Make Spectrogram for each audio file

In [ ]:
filename_metadata = "data/fma_metadata/tracks.csv"
tracks = pd.read_csv(filename_metadata, header=2, low_memory=False)
tracks_array = tracks.values
tracks_id_array = tracks_array[: , 0]
tracks_genre_array = tracks_array[: , 40]
tracks_id_array = tracks_id_array.reshape(tracks_id_array.shape[0], 1)
tracks_genre_array = tracks_genre_array.reshape(tracks_genre_array.shape[0], 1)

folder_sample = "data/fma_small"
directories = [d for d in os.listdir(folder_sample)
               if os.path.isdir(os.path.join(folder_sample, d))]

counter = 0
if not os.path.exists('data/Train_Spectogram_Images_big'):
    os.makedirs('data/Train_Spectogram_Images_big')
for d in directories:
    label_directory = os.path.join(folder_sample, d)
    file_names = [os.path.join(label_directory, f)
                  for f in os.listdir(label_directory)
                  if f.endswith(".mp3")]
    # Convert .mp3 files into mel-Spectograms
    for f in file_names:
        try:
            track_id = f[-10:-4]
            tracks_id_array1 = [str(s[0]).zfill(6) for s in tracks_id_array]
            track_index = list(tracks_id_array1).index(str(track_id))
            y, sr = librosa.load(f)
            melspectrogram_array = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
            mel = librosa.power_to_db(melspectrogram_array)
            # Length and Width of Spectogram
            fig_size = plt.rcParams["figure.figsize"]
            fig_size[0] = float(mel.shape[1]) / float(50)
            fig_size[1] = float(mel.shape[0]) / float(50)
            plt.rcParams["figure.figsize"] = fig_size
            plt.axis('off')
            plt.axes([0., 0., 1., 1.0], frameon=False, xticks=[], yticks=[])
            librosa.display.specshow(mel, cmap='gray_r')
            plt.savefig("data/Train_Spectogram_Images_big/"+str(track_id)+"_"+str(tracks_genre_array[track_index,0])+".jpg", bbox_inches=None, pad_inches=0)
            plt.close()
            counter = counter + 1
        except:
            continue

## Slice spectrograms into x-sec chunks

In [21]:
# Change frame_length below to change the length of slices
def slice_mel_spectrogram(mel_spectrogram):
    mel_spectrogram = mel_spectrogram[:,:]  # extract only one channel (e.g., the first channel)
    frame_length = 256
    spectrogram_length = mel_spectrogram.shape[1]
    num_frames = int(np.floor(spectrogram_length / frame_length))
    frames = np.zeros((num_frames, mel_spectrogram.shape[0], frame_length))
    for i in range(num_frames):
        start = i * frame_length
        end = min(start + frame_length, spectrogram_length)
        frames[i,:,:end-start] = mel_spectrogram[:,start:end]
    return frames

image_folder = "data/Train_Spectogram_Images_big"
filenames = [os.path.join(image_folder, f) for f in os.listdir(image_folder)
               if f.endswith(".jpg")]
if not os.path.exists('data/Train_Sliced_Images_big'):
    os.makedirs('data/Train_Sliced_Images_big')
for f in filenames:
    track_id = f.split('\\')[-1].split('_')[0]
    genre_variable = re.search(r"_([^_]+)\.jpg$", f).group(1)
    img = Image.open(f).convert('L')
    mel_spectrogram = np.array(img)
    frames = slice_mel_spectrogram(mel_spectrogram)
    for i, frame in enumerate(frames):
        img_temporary = Image.fromarray(frame).convert('L')
        img_temporary.save("data/Train_Sliced_Images_big/"+"_"+track_id+'_'+str(i)+'_'+genre_variable+".jpg")

In [8]:
genre = {
"Hip-Hop": 0,
"International": 1,
"Electronic": 2,
"Folk" : 3,
"Experimental": 4,
"Rock": 5,
"Pop": 6,
"Instrumental": 7
}

filenames = [os.path.join("data/Train_Sliced_Images_big", f) for f in os.listdir("data/Train_Sliced_Images_big")
               if f.endswith(".jpg")]
images_all = [None]*(len(filenames))
labels_all = [None]*(len(filenames))
for i,f in enumerate(filenames):
    index = i
    genre_variable = re.search(r"_([^_]+)\.jpg$", f).group(1)
    temp = cv2.imread(f, cv2.IMREAD_UNCHANGED)
    images_all[index] = temp
    labels_all[index] = genre[genre_variable]

images = np.array(images_all)
labels = np.array(labels_all)
labels = labels.reshape(labels.shape[0],1)
train_x, test_x, train_y, test_y = train_test_split(images, labels, test_size=0.05, shuffle=True)
# Convert the labels into one-hot vectors.
train_y = np_utils.to_categorical(train_y)
test_y = np_utils.to_categorical(test_y, num_classes=8)


In [9]:
train_x.shape, train_y.shape

((4750, 256, 256), (4750, 8))

In [25]:

n_classes = len(genre)
genre_new = {value: key for key, value in genre.items()}

if not os.path.exists('data/Training_Data'):
    os.makedirs('data/Training_Data')
np.save("data/Training_Data/train_x.npy", train_x)
np.save("data/Training_Data/train_y.npy", train_y)
np.save("data/Training_Data/test_x.npy", test_x)
np.save("data/Training_Data/test_y.npy", test_y)

In [26]:
train_x = np.load("data/Training_Data/train_x.npy")
train_y = np.load("data/Training_Data/train_y.npy")
test_x = np.load("data/Training_Data/test_x.npy")
test_y = np.load("data/Training_Data/test_y.npy")

In [10]:
# Expand the dimensions of the image to have a channel dimension. (nx128x128) ==> (nx128x128x1)
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], 1)

In [11]:
# Normalize the matrices.
train_x = train_x / 255.
test_x = test_x / 255.

In [12]:
train_y.shape

(4750, 8)

In [14]:
# Create Model
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=[7,7], kernel_initializer = initializers.he_normal(seed=1), 
                 activation="relu", input_shape=(128,128,1)))
# Dim = (122x122x64)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (61x61x64)
model.add(Conv2D(filters=128, kernel_size=[7,7], strides=2, kernel_initializer = initializers.he_normal(seed=1), 
                 activation="relu"))
# Dim = (28x28x128)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (14x14x128)

model.add(BatchNormalization())
model.add(Flatten())
# Dim = (2048)
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(1024, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (1024)
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (256)
model.add(Dropout(0.25))
model.add(Dense(64, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (64)
model.add(Dense(32, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (32)
model.add(Dense(train_y.shape[1], activation="softmax", kernel_initializer=initializers.he_normal(seed=1)))

In [17]:
model.summary()
plot_model(model, to_file="Saved_Model/Model_Architecture.jpg")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 122, 424, 64)      3200      
                                                                 
 batch_normalization_4 (Batc  (None, 122, 424, 64)     256       
 hNormalization)                                                 
                                                                 
 average_pooling2d_2 (Averag  (None, 61, 212, 64)      0         
 ePooling2D)                                                     
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 103, 128)      401536    
                                                                 
 batch_normalization_5 (Batc  (None, 28, 103, 128)     512       
 hNormalization)                                                 
                                                      

In [61]:
model.compile(loss="categorical_crossentropy", 
              optimizer= Adam(learning_rate=0.0001), 
              metrics=['accuracy'])

In [ ]:
model_hist = model.fit(train_x, train_y, epochs=10, verbose=1, validation_split=0.1)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=1)
print score
model.save("Saved_Model/Model.h5")

In [ ]:
def plot_history(hist):
    """Plots the accuracy and loss for a model over the course of all epochs
    
    Parameters:
        hist (keras history object): The recorded history of model.fit() to be plotted
    """
    fig, axs = plt.subplots(2, 1, figsize=(8,7))
    fig.tight_layout(pad=2)
    
    # Accuracy subplot
    axs[0].plot(hist.history["acc"], c='navy', label="Training Accuracy")
    axs[0].plot(hist.history["val_acc"], c='orange', label="Validation Accuracy")    
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy")
    
    # Error subplot
    axs[1].plot(hist.history["loss"], c='navy', label="Training Loss")
    axs[1].plot(hist.history["val_loss"], c='orange', label="Validation Loss")    
    axs[1].set_ylabel("Loss")
    axs[1].set_xlabel("Epochs")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Loss")
    
    plt.show()

In [ ]:
plot_history(model_hist)